# [TRAIN] 2 classifier

[[Notebook] Reference Notebook for 2 classifier](https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-fold0-5-2class?select=train.csv)

what is 2 classifier?  
either ***object(box)*** or ***Not***   
image level 에서 none 과 그렇지 않은 것을 구분하는 classifier.  
왜 그렇게 하는가? 조사  

1. efficientnet b7 kfold traininig on TPU
2. efficientnetV2_large [[Notebook] Reference Notebook for using EffNetV2](https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-keras-study-train-tpu-cv0-805/notebook)

-> 둘 다 인터넷 되므로 패키지 설치할 필요 없다.

추후에 이미지 resize 생성시, resolution 상승시켜 학습도 해야겠다.   

----

### [[Discussion] host - annotation](https://www.kaggle.com/c/siim-covid19-detection/discussion/246597)  

테스트 데이터 세트에 대해서는 label 업데이트. but, train label 은 그대로 유지.  

test data label 은 수정되었으나, train data label 은 시간 제약으로 동일하게 유지되고 업데이트 되지 않았다.  

→ 따라서 연구에 속한 이미지가 2개 이상인 경우, bounding boxes 가 있는 이미지만 사용하는 것을 추천한다.  

**즉, 데이터셋을 아예 새롭게 구성한다.**

위와 같이 dataset 중 study_level 마다 이미지가 2개이상인 데이터의 경우, 같은 이미지임에도 box가 있고 없는 이미지들이 있다.  
test set 은 추후에 label annotating 보정이 되었기 때문에, train data 에서 제외시켜야 한다.   
또한 이러한 이미지들은 none 여부에 영향을 미치므로 detection 만이 아니라 2 class 도 다시 학습시켜준다.  
study_level 도 다시 학습 시킬 예정이긴 하나, study_level 은 study 단위이므로 큰 의미는 없을 것이다.
그렇게 다시 efnb7 부터 학습을 다시 시행해본다.  

### Dataset GCS path 
Kaggle에서 Dataset 생성시 바로 download가 안될때 보면 gcs path 가 없다고 나온다.  
아마 이전에 계속 오류가 뜬 부분이 여기가 아닌가 싶다.  
이전에 private 형태로 사용하기 위한 절차를 수행했는데, 그 과정에서 문제가 생긴게 아닌가 싶다.  
Dataset 을 재생성했고, gcs path 가 생성된 것을 확인 후 run -> ok

### TPU for Private Datasets
[[Discussion] TPU for Private Datasets](https://www.kaggle.com/product-feedback/163416#1298583)  
[[Article] How to use TPU](https://www.kaggle.com/docs/tpu)

In [1]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# user_credential = user_secrets.get_gcloud_credential()
# user_secrets.set_tensorflow_credential(user_credential)

In [2]:
!pip install efficientnet -q

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import efficientnet.tfkeras as efn
# use TPU by kaggle
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import shutil

from glob import glob
import itertools
import random
import gc
import math
import pprint
import seaborn as sns
import tensorflow_hub as tfhub
import tensorflow.keras.backend as K

import albumentations
from PIL import Image, ImageOps, ImageEnhance
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.augmentations import functional as F

print('TF version:', tf.__version__)
print('Hub version:', tfhub.__version__)
print('Physical devices:', tf.config.list_physical_devices())

TF version: 2.4.1
Hub version: 0.12.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
# Config for EfficientNetV2

v1 = False
v2 = True

def seed_everything(SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)

print(f"Use model Version : {'V1' if v1 else 'V2'}")
seed_everything(42)

Use model Version : V2


In [4]:
# original Notebook (https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-keras-study-train-tpu-cv0-805)
# def get_mat : transform matrix
# def transform : augmentation

In [5]:
SATURATION  = (0.9, 1.1)
CONTRAST = (0.9, 1.1)
BRIGHTNESS  =  0.1
ROTATION    = 10.0
SHEAR    = 2.0
HZOOM  = 8.0
WZOOM  = 4.0
HSHIFT = 4.0
WSHIFT = 4.0

def auto_select_accelerator():
    """
        자동으로 가속기 선택. 
        조건이 될 때는 TPU. 
        조건이 안될때는 할당된 가속기 선택.
    """
    # Detect hardware, return appropriate distribution strategy
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu) # default distribution strategy in Tensorflow. Works on CPU and single GPU.
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
#        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, SATURATION[0], SATURATION[1])
        img = tf.image.random_contrast(img, CONTRAST[0], CONTRAST[1])
        img = tf.image.random_brightness(img, BRIGHTNESS)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment
# 다른 Notebook 에서는 flip_up_down 을 지우고(하긴 위아래가 바뀔 일을 없으니)
# saturation, contrast, brightness 를 추가하였음.

def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None, do_mix = True,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset # augmentation
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    dset = dset.map(transform, num_parallel_calls=AUTO) if do_mix else dset # cutmix
    return dset

print("Load Fucntion")

Load Fucntion


## Mixed Precision and/or XLA

Mixed Precision and XLA are not being used in this notebook but you can experiment using them. Change the following booleans to enable mixed precision and/or XLA on GPU/TPU. By default TPU already uses some mixed precision but we can add more (and it already uses XLA). These allow the GPU/TPU memory to handle larger batch sizes and can speed up the training process. The Nvidia V100 GPU has special Tensor Cores which get utilized when mixed precision is enabled. Unfortunately Kaggle's Nvidia P100 GPU does not have Tensor Cores to receive speed up.

In [6]:
MIXED_PRECISION = False
XLA_ACCELERATE = False

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

In [7]:
#COMPETITION_NAME = 'siimcovid19-512-img-png-600-study-png'
COMPETITION_NAME = 'siim-image'
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
df = pd.read_csv("/kaggle/input/siim-image/new_resized_data/new_df.csv")
df['none_or_box'] = df['label'].apply(lambda x: 1 if x.split()[0] == 'none' else 0)
df[['none','box']] = tf.one_hot(df['none_or_box'].values, 2, dtype = tf.int8)
label_cols = df.columns[5:7]
print(label_cols)

# kfold split
gkf = GroupKFold(n_splits = 5)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups=df.StudyInstanceUID.tolist())):
    df.loc[val_idx, 'fold'] = fold
    
display(df.head(), df.shape)

Index(['none', 'box'], dtype='object')


,id,boxes,label,StudyInstanceUID,none_or_box,none,box,fold
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,1,2
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,4
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,1,0,1
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0


(6117, 8)

In [8]:
%%time
#training

if v1:
    trained_model_path = '/kaggle/working/effnetB7_model'
    if not os.path.isdir(trained_model_path):
        os.makedirs(trained_model_path)
    else:
        shutil.rmtree(trained_model_path)

if v1:
    GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME) # dataset path for all
    # COMPETITION_NAME 으로 지정한 public data folder 를 kaggle datasets 을 활용해서 
    # google cloud storage 에 연결하는 코드 -> for using TPU
    strategy = auto_select_accelerator()
    BATCH_SIZE = strategy.num_replicas_in_sync * 16
    

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs


## Custom LR scheduler

In [9]:
# Configuration
# Configuration
configurating = False
scheduling = False

if configurating:
    IMAGE_SIZE = [512, 512]
    EPOCHS = 20
    FOLDS = 3
    SEED = 777
    BATCH_SIZE = 16 * strategy.num_replicas_in_sync
    AUG_BATCH = BATCH_SIZE
    FIRST_FOLD_ONLY = False

if scheduling:
    # Learning rate schedule for TPU, GPU and CPU.
    # Using an LR ramp up because fine-tuning a pre-trained model.
    # Starting with a high LR would break the pre-trained weights.
    
    EPOCHS = 20
    
    LR_START = 0.00005
    LR_MAX = 0.00002 * strategy.num_replicas_in_sync # 8
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8

    def lrfn(epoch):
        if epoch < LR_RAMPUP_EPOCHS:
            lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
        elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
            lr = LR_MAX
        else:
            lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

    rng = [i for i in range(20 if EPOCHS < 20 else EPOCHS)]
    y = [lrfn(x) for x in rng]
    plt.plot(rng, y)
    print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

# CutMix Augmentation
The following code does cutmix using the GPU/TPU. Change the variables `SWITCH`, `CUTMIX_PROB` and `MIXUP_PROB` in function `transform()` to control the amount of augmentation during training. CutMix will occur `SWITCH * CUTMIX_PROB` often and MixUp will occur `(1-SWITCH) * MIXUP_PROB` often during training.

[[Notebook] CutMix and MixUp on GPU/TPU](https://www.kaggle.com/cdeotte/cutmix-and-mixup-on-gpu-tpu)

In [10]:
def onehot(image, label):
    CLASSES = 2
    return image, tf.one_hot(label, CLASSES)

# only implement cutmix for testing because
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n, dim, dim, 3] not a single image of [dim, dim, 3]
    # output - a batch of images with cutmix applied
    DIM = 512
    CLASSES = 2
    AUG_BATCH = 128 if v1 else 64
    imgs = []
    labs = []

    for j in range(AUG_BATCH): # = BATCH_SIZE
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([],0,DIM), tf.int32)
        y = tf.cast(tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM, tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = tf.cast(label[j,], tf.float32)
            lab2 = tf.cast(label[k,], tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
        
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2, label2

def transform(image, label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = 512
    CLASSES = 2
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    AUG_BATCH = 128 if v1 else 64
        
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    imgs = []
    labs = []
    for j in range(AUG_BATCH):
        #P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(image2[j,])
        labs.append(label2[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR
    image4 = tf.reshape(tf.stack(imgs), (AUG_BATCH, DIM, DIM, 3))
    label4 = tf.reshape(tf.stack(labs), (AUG_BATCH, CLASSES))
    return image4, label4

# Display CutMix Augmentation

In [11]:
display_image = False

if display_image:
    i = 0
    detail_path = '/new_resized_data/image_512/'
    valid_paths = GCS_DS_PATH + detail_path + df[df['fold'] == i]['id'] + '.png'
    train_paths = GCS_DS_PATH + detail_path + df[df['fold'] != i]['id'] + '.png'
    valid_labels = df[df['fold'] == i][label_cols].values
    train_labels = df[df['fold'] != i][label_cols].values
    img_size = 512
    BATCH_SIZE = 128
    # train image
    decoder = build_decoder(with_labels=True,
                            target_size=(img_size, img_size),
                            ext='png')
    # valid image
    test_decoder = build_decoder(with_labels=False, 
                                 target_size=(img_size, img_size),
                                 ext='png')

    train_dataset = build_dataset(train_paths,
                                  train_labels,
                                  bsize=BATCH_SIZE,
                                  decode_fn=decoder)

    valid_dataset = build_dataset(valid_paths,
                                  valid_labels,
                                  bsize=BATCH_SIZE,
                                  decode_fn=decoder,
                                  repeat=False,
                                  shuffle=False,
                                  augment=False,
                                  do_mix=False)
    # iterator 가 아니다. repeat() 로 batch 를 읽는다.
    sample = train_dataset.repeat()
    for (img, label) in sample:
        train_img = img[:16]
        train_label = label[:16]
        break
    print('load train dataset')
    sample = valid_dataset.repeat()
    for (img, label) in sample:
        valid_img = img[:16]
        valid_label = label[:16]
        break
    print('load valid dataset')

In [12]:
if display_image:
    fig, axes = plt.subplots(4,4, figsize=(12,12))
    for i in range(16):
        axes[i//4][i%4].imshow(train_img[i])
        axes[i//4][i%4].set_title(f"{train_label[i].numpy()[1]}")
    plt.show();
    
    fig, axes = plt.subplots(4,4, figsize=(12,12))
    for i in range(16):
        axes[i//4][i%4].imshow(valid_img[i])
        axes[i//4][i%4].set_title(f"{valid_label[i].numpy()[1]}")
    plt.show();

# AugMix Augmentation
[[Notebook] AugMix Data augmentation on TPU](https://www.kaggle.com/szacho/augmix-data-augmentation-on-tpu)   

AugMix 는 구현 실패

In [13]:
# Augmix Helper Funtion
def int_parameter(level, maxval):
    return tf.cast(level * maxval / 10, tf.int32)

def float_parameter(level, maxval):
    return tf.cast((level) * maxval / 10., tf.float32)

def sample_level(n):
    return tf.random.uniform(shape=[1], minval=0.1, maxval=n, dtype=tf.float32)
    
def affine_transform(image, transform_matrix):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    x = tf.repeat(tf.range(DIM//2,-DIM//2,-1), DIM)
    y = tf.tile(tf.range(-DIM//2,DIM//2), [DIM])
    z = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack([x, y, z])
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(transform_matrix, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d = tf.gather_nd(image, tf.transpose(idx3))
    return tf.reshape(d,[DIM,DIM,3])

def blend(image1, image2, factor):
    if factor == 0.0:
        return tf.convert_to_tensor(image1)
    if factor == 1.0:
        return tf.convert_to_tensor(image2)

    image1 = tf.cast(image1, tf.float32)
    image2 = tf.cast(image2, tf.float32)

    difference = image2 - image1
    scaled = factor * difference

    # Do addition in float.
    temp = tf.cast(image1, tf.float32) + scaled

    # Interpolate
    if factor > 0.0 and factor < 1.0:
        # Interpolation means we always stay within 0 and 255.
        return tf.cast(temp, tf.uint8)

    # Extrapolate:
    #
    # We need to clip and then cast.
    return tf.cast(tf.clip_by_value(temp, 0.0, 255.0), tf.uint8)

### Transformations
These are simple augmentations used by AugMix. Every function takes `image` and `level` (integer from 1 to 10) as arguments. The second one indicates how much variation will particular transformation yield, in other words, how strong it will be.

Translate, shear and rotate augmentations are based on [this notebook](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96).

In [14]:
def rotate(image, level):
    degrees = float_parameter(sample_level(level), 30)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    degrees = tf.cond(rand_var > 0.5, lambda: degrees, lambda: -degrees)

    angle = math.pi*degrees/180 # convert degrees to radians
    angle = tf.cast(angle, tf.float32)
    # define rotation matrix
    c1 = tf.math.cos(angle)
    s1 = tf.math.sin(angle)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape(tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one], axis=0), [3,3])

    transformed = affine_transform(image, rotation_matrix)
    return transformed

def translate_x(image, level):
    lvl = int_parameter(sample_level(level), IMAGE_SIZE[0] / 3)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    lvl = tf.cond(rand_var > 0.5, lambda: lvl, lambda: -lvl)

    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    lvl = tf.cast(lvl, tf.float32)
    translate_x_matrix = tf.reshape(tf.concat([one,zero,zero, zero,one,lvl, zero,zero,one], axis=0), [3,3])

    transformed = affine_transform(image, translate_x_matrix)
    return transformed

def translate_y(image, level):
    lvl = int_parameter(sample_level(level), IMAGE_SIZE[0] / 3)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    lvl = tf.cond(rand_var > 0.5, lambda: lvl, lambda: -lvl)

    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    lvl = tf.cast(lvl, tf.float32)
    translate_y_matrix = tf.reshape(tf.concat([one,zero,lvl, zero,one,zero, zero,zero,one], axis=0), [3,3])

    transformed = affine_transform(image, translate_y_matrix)
    return transformed

def shear_x(image, level):
    lvl = float_parameter(sample_level(level), 0.3)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    lvl = tf.cond(rand_var > 0.5, lambda: lvl, lambda: -lvl)

    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    s2 = tf.math.sin(lvl)
    shear_x_matrix = tf.reshape(tf.concat([one,s2,zero, zero,one,zero, zero,zero,one],axis=0), [3,3])   

    transformed = affine_transform(image, shear_x_matrix)
    return transformed

def shear_y(image, level):
    lvl = float_parameter(sample_level(level), 0.3)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    lvl = tf.cond(rand_var > 0.5, lambda: lvl, lambda: -lvl)

    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')
    c2 = tf.math.cos(lvl)
    shear_y_matrix = tf.reshape(tf.concat([one,zero,zero, zero,c2,zero, zero,zero,one],axis=0), [3,3])   
    
    transformed = affine_transform(image, shear_y_matrix)
    return transformed

def solarize(image, level):
    # For each pixel in the image, select the pixel
    # if the value is less than the threshold.
    # Otherwise, subtract 255 from the pixel.
    threshold = float_parameter(sample_level(level), 1)
    return tf.where(image < threshold, image, 1 - image)

def solarize_add(image, level):
    # For each pixel in the image less than threshold
    # we add 'addition' amount to it and then clip the
    # pixel value to be between 0 and 255. The value
    # of 'addition' is between -128 and 128.
    threshold = float_parameter(sample_level(level), 1)
    addition = float_parameter(sample_level(level), 0.5)
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    addition = tf.cond(rand_var > 0.5, lambda: addition, lambda: -addition)

    added_image = tf.cast(image, tf.float32) + addition
    added_image = tf.cast(tf.clip_by_value(added_image, 0, 1), tf.float32)
    return tf.where(image < threshold, added_image, image)

def posterize(image, level):
    lvl = int_parameter(sample_level(level), 8)
    shift = 8 - lvl
    shift = tf.cast(shift, tf.uint8)
    image = tf.cast(tf.math.scalar_mul(255, image), tf.uint8)
    image = tf.bitwise.left_shift(tf.bitwise.right_shift(image, shift), shift)
    return tf.cast(tf.clip_by_value(tf.math.divide(image, 255), 0, 1), tf.float32)

def autocontrast(image, _):
    image = tf.cast(tf.math.scalar_mul(255, image), tf.uint8)

    def scale_channel(image):
        # A possibly cheaper version can be done using cumsum/unique_with_counts
        # over the histogram values, rather than iterating over the entire image.
        # to compute mins and maxes.
        lo = tf.cast(tf.reduce_min(image), tf.float32)
        hi = tf.cast(tf.reduce_max(image), tf.float32)

        # Scale the image, making the lowest value 0 and the highest value 255.
        def scale_values(im):
            scale = 255.0 / (hi - lo)
            offset = -lo * scale
            im = tf.cast(im, tf.float32) * scale + offset
            im = tf.clip_by_value(im, 0.0, 255.0)
            return tf.cast(im, tf.uint8)

        result = tf.cond(hi > lo, lambda: scale_values(image), lambda: image)
        return result

    # Assumes RGB for now.  Scales each channel independently
    # and then stacks the result.
    s1 = scale_channel(image[:, :, 0])
    s2 = scale_channel(image[:, :, 1])
    s3 = scale_channel(image[:, :, 2])
    image = tf.stack([s1, s2, s3], 2)
    return tf.cast(tf.clip_by_value(tf.math.divide(image, 255), 0, 1), tf.float32)

def equalize(image, _):
    image = tf.cast(tf.math.scalar_mul(255, image), tf.uint8)

    def scale_channel(im, c):
        im = tf.cast(im[:, :, c], tf.int32)
        # Compute the histogram of the image channel.
        histo = tf.histogram_fixed_width(im, [0, 255], nbins=256)
        # For the purposes of computing the step, filter out the nonzeros.
        nonzero = tf.where(tf.not_equal(histo, 0))
        nonzero_histo = tf.reshape(tf.gather(histo, nonzero), [-1])
        step = (tf.reduce_sum(nonzero_histo) - nonzero_histo[-1]) // 255

        def build_lut(histo, step):
            # Compute the cumulative sum, shifting by step // 2
            # and then normalization by step.
            lut = (tf.cumsum(histo) + (step // 2)) // step
            # Shift lut, prepending with 0.
            lut = tf.concat([[0], lut[:-1]], 0)
            # Clip the counts to be in range.  This is done
            # in the C code for image.point.
            return tf.clip_by_value(lut, 0, 255)

        # If step is zero, return the original image.  Otherwise, build
        # lut from the full histogram and step and then index from it.
        result = tf.cond(tf.equal(step, 0),
                        lambda: im,
                        lambda: tf.gather(build_lut(histo, step), im))

        return tf.cast(result, tf.uint8)

    # Assumes RGB for now.  Scales each channel independently
    # and then stacks the result.
    s1 = scale_channel(image, 0)
    s2 = scale_channel(image, 1)
    s3 = scale_channel(image, 2)
    image = tf.stack([s1, s2, s3], 2)

    return tf.cast(tf.clip_by_value(tf.math.divide(image, 255), 0, 1), tf.float32)

def color(image, level):
    factor = float_parameter(sample_level(level), 1.8) + 0.1
    image = tf.cast(tf.math.scalar_mul(255, image), tf.uint8)
    degenerate = tf.image.grayscale_to_rgb(tf.image.rgb_to_grayscale(image))
    blended = blend(degenerate, image, factor)
    return tf.cast(tf.clip_by_value(tf.math.divide(blended, 255), 0, 1), tf.float32)

def brightness(image, level):
    delta = float_parameter(sample_level(level), 0.5) + 0.1
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    delta = tf.cond(rand_var > 0.5, lambda: delta, lambda: -delta) 
    return tf.image.adjust_brightness(image, delta=delta)

def contrast(image, level):
    factor = float_parameter(sample_level(level), 1.8) + 0.1
    factor = tf.reshape(factor, [])
    rand_var = tf.random.uniform(shape=[], dtype=tf.float32)
    factor = tf.cond(rand_var > 0.5, lambda: factor, lambda: 1.9 - factor  )

    return tf.image.adjust_contrast(image, factor)


### AugMix

In [15]:
means = {'R': 0.44892993872313053, 'G': 0.4148519066242368, 'B': 0.301880284715257}
stds = {'R': 0.24393544875614917, 'G': 0.2108791383467354, 'B': 0.220427056859487}

def substract_means(image):
    image = image - np.array([means['R'], means['G'], means['B']])
    return image

def normalize(image):
    image = substract_means(image)
    image = image / np.array([stds['R'], stds['G'], stds['B']])
    return tf.clip_by_value(image, 0, 1)

def apply_op(image, level, which):
    # is there any better way than manually typing all of these conditions? 
    # I tried to randomly select transformation from array of functions, but tensorflow didn't let me to
    augmented = image
    augmented = tf.cond(which == tf.constant([0], dtype=tf.int32), lambda: rotate(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([1], dtype=tf.int32), lambda: translate_x(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([2], dtype=tf.int32), lambda: translate_y(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([3], dtype=tf.int32), lambda: shear_x(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([4], dtype=tf.int32), lambda: shear_y(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([5], dtype=tf.int32), lambda: solarize_add(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([6], dtype=tf.int32), lambda: solarize(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([7], dtype=tf.int32), lambda: posterize(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([8], dtype=tf.int32), lambda: autocontrast(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([9], dtype=tf.int32), lambda: equalize(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([10], dtype=tf.int32), lambda: color(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([11], dtype=tf.int32), lambda: contrast(image, level), lambda: augmented)
    augmented = tf.cond(which == tf.constant([12], dtype=tf.int32), lambda: brightness(image, level), lambda: augmented)
    return augmented

def augmix(image):
    # you can play with these parameters
    severity = 3 # level of transformations as described above in transformations (integer from 1 to 10)
    width = 3 # number of different chains of transformations to be mixed
    depth = -1 # number of transformations in one chain, -1 means random from 1 to 3
    
    alpha = 1.
    dir_dist = tfp.distributions.Dirichlet([alpha]*width)
    ws = tf.cast(dir_dist.sample(), tf.float32)
    beta_dist = tfp.distributions.Beta(alpha, alpha)
    m = tf.cast(beta_dist.sample(), tf.float32)

    mix = tf.zeros_like(image, dtype='float32')

    def outer_loop_cond(i, depth, mix):
        return tf.less(i, width)

    def outer_loop_body(i, depth, mix):
        image_aug = tf.identity(image)
        depth = tf.cond(tf.greater(depth, 0), lambda: depth, lambda: tf.random.uniform(shape=[], minval=1, maxval=3, dtype=tf.int32))

        def inner_loop_cond(j, image_aug):
            return tf.less(j, depth)

        def inner_loop_body(j, image_aug):
            which = tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32)
            image_aug = apply_op(image_aug, severity, which)
            j = tf.add(j, 1)
            return j, image_aug
        
        j = tf.constant([0], dtype=tf.int32)
        j, image_aug = tf.while_loop(inner_loop_cond, inner_loop_body, [j, image_aug])

        wsi = tf.gather(ws, i)
        mix = tf.add(mix, wsi*normalize(image_aug))
        i = tf.add(i, 1)
        return i, depth, mix

    i = tf.constant([0], dtype=tf.int32)
    i, depth, mix = tf.while_loop(outer_loop_cond, outer_loop_body, [i, depth, mix])
    
    mixed = tf.math.scalar_mul((1 - m), normalize(image)) + tf.math.scalar_mul(m, mix)
    return tf.clip_by_value(mixed, 0, 1)

# Build, Train, Infer Model
This is the 5-Fold workflow copied from Ragnar's notebook here. Now we add cutmix and/or mixup augmentation to the training images on the fly! Since cutmix and/or mixup returns one hot encoded labels, we change the loss and metric to `categorical_crossentropy` instead of `sparse_categorical_crossentropy`.



# EfficientNetB7

In [16]:
ONE_FOLD = False
if v1:
    for i in range(5):
        print(f"\n ---- Fold {i} start ---- \n")
        detail_path = '/new_resized_data/image_512/'
        valid_paths = GCS_DS_PATH + detail_path + df[df['fold'] == i]['id'] + '.png'
        train_paths = GCS_DS_PATH + detail_path + df[df['fold'] != i]['id'] + '.png'
        valid_labels = df[df['fold'] == i][label_cols].values
        train_labels = df[df['fold'] != i][label_cols].values

        IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
        IMS = 8 # image_level file 을 사용하므로 img_size = 512
        
        
        decoder = build_decoder(with_labels=True,
                                target_size=(IMSIZE[IMS], IMSIZE[IMS]),
                                ext='png')
        
        test_decoder = build_decoder(with_labels=False, 
                                     target_size=(IMSIZE[IMS], IMSIZE[IMS]),
                                     ext='png')

        train_dataset = build_dataset(train_paths,
                                      train_labels,
                                      bsize=BATCH_SIZE,
                                      decode_fn=decoder)
        
        valid_dataset = build_dataset(valid_paths,
                                      valid_labels,
                                      bsize=BATCH_SIZE,
                                      decode_fn=decoder,
                                      repeat=False,
                                      shuffle=False,
                                      augment=False,
                                      do_mix=False)
        try:
            n_labels = train_labels.shape[1]
        except:
            n_labels = 1
        print(f"\n n_labels : {n_labels} \n")
        with strategy.scope():
            model = tf.keras.Sequential([
                efn.EfficientNetB7(input_shape = (IMSIZE[IMS], IMSIZE[IMS], 3),
                                   weights = 'imagenet',
                                   include_top=False),
                tf.keras.layers.GlobalAveragePooling2D(),
                tf.keras.layers.Dense(n_labels, activation='softmax')
            ])
            model.compile(optimizer = tf.keras.optimizers.Adam(),
                          loss='categorical_crossentropy', # 'binary_crossentropy'
                          metrics = [tf.keras.metrics.AUC(multi_label=True)])
            model.summary()

        steps_per_epoch = train_paths.shape[0] // BATCH_SIZE 
        checkpoint = tf.keras.callbacks.ModelCheckpoint(f'{trained_model_path}/model{i}_2class.h5', 
                                                        save_best_only=True, 
                                                        monitor='val_loss',
                                                        mode='min')
        
        lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          patience=3,
                                                          min_lr=1e-6,
                                                          factor=0.2,
                                                          mode='min')
        
        annealing = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.0001,
                                                              first_decay_steps=10,
                                                              t_mul=1.0,
                                                              m_mul=1.0,
                                                              alpha=0.0)
        lr_callback = tf.keras.callbacks.LearningRateScheduler(annealing, verbose = True)
        
        history = model.fit(train_dataset,
                            epochs=20,
                            verbose=1,
                            callbacks=[checkpoint, lr_reducer, annealing],
                            steps_per_epoch=steps_per_epoch,
                            validation_data=valid_dataset)
        hist_df = pd.DataFrame(history.history)
        hist_df.to_csv(f"{trained_model_path}/history{i}_2class.csv")
        if ONE_FOLD:
            break

In [17]:
# comment
# target val_AUC : 0.892, val_loss : 0.394 (ratio 2 with LB)

In [18]:
if False:
    history_lst = sorted(glob('./effnetB7_model/*.csv'))
    loss_lst = []
    auc_lst = []
    for i, his in enumerate(history_lst):
        tmp = pd.read_csv(his)
        tmp = tmp[tmp['val_loss']==tmp['val_loss'].min()]
        loss_lst.append(tmp['val_loss'].values)
        t = 'val_auc' if i == 0 else f"val_auc_{i}"
        auc_lst.append(tmp[t])
    print(loss_lst, np.mean(loss_lst))
# cutmix : 0.394 -> 0.387, auc 는 더 하락 -> LB 엄청 하락? why? AUC 기준으로 생각하자.

# Validation

In [19]:
predict_valid = False
if predict_valid:
    model_lst = []
    for i in range(5):
        model = tf.keras.models.load_model(
            f'../input/effnetb7-cutmix/kaggle/working/effnetB7_model/model{i}_2class.h5'
            )
        model_lst.append(model)
    print(model_lst)

In [20]:
if predict_valid:
    # valid image
    img_size = 512
    decoder = build_decoder(with_labels=False, 
                            target_size=(img_size, img_size),
                            ext='png')
    all_dataset = []
    all_labels = []
    for i in range(5):
        print(f"\n ---- Fold {i} start ---- \n")
        detail_path = '/new_resized_data/image_512/'
        valid_paths = GCS_DS_PATH + detail_path + df[df['fold'] == i]['id'] + '.png'
        valid_labels = df[df['fold'] == i][label_cols].values
        
        valid_dataset = build_dataset(valid_paths, # without 
                                      bsize=BATCH_SIZE,
                                      decode_fn=decoder,
                                      repeat=False,
                                      shuffle=False,
                                      augment=False,
                                      do_mix=False)
        all_dataset.append(valid_dataset)
        all_labels.append(valid_labels)

In [21]:
if predict_valid:
    scores = []
    for i, model in enumerate(model_lst):
        print(f"model {i} start")
        for valid_data, valid_label in zip(all_dataset, all_labels):
            result = model.predict(valid_data)[:,1]
            sample_score = roc_auc_score(valid_label[:,1], result)
            scores.append(sample_score)

In [22]:
if predict_valid:
    result_df = pd.DataFrame({'pred':result[:,1],
                              'label':valid_label[:,1]})
    result_df.to_csv("result_df.csv", index=False)
    zero = result_df[result_df['label']==0]
    one = result_df[result_df['label']==1]
    plt.hist(zero['pred'], bins=20, alpha=0.3, label='pred for answer zero')
    plt.hist(one['pred'], bins=20, alpha=0.3, label='pred for answer one')
    plt.show();
    
    plt.figure(figsize=(14,6))
    for col in hist_df.columns:
        value = hist_df[col].values.copy()
        if col == 'lr':
            value *= 100
            col = f"{col} * 100"
        plt.plot(value, alpha=0.6, marker='o', markersize=3, label=col)
    plt.legend()
    plt.grid()
    plt.show();
    
    del model

# EfficientNetV2_XL

In [23]:
# We first find the GCS path of the selected EffNetV2 architecture from the EffNetV2 weights Kaggle dataset

if v2:
    # Get the Tensorflow Hub model URL
    hub_type = 'feature_vector' # ['classification', 'feature_vector']
    model_arch = "efficientnetv2-l-21k-ft1k"
    # Get the GCS path of EfficientNet Models
    DS_GCS_PATH = KaggleDatasets().get_gcs_path('efficientnetv2-tfhub-weight-files')
    MODEL_GCS_PATH = f"{DS_GCS_PATH}/tfhub_models/{model_arch}/{hub_type}"
    print(MODEL_GCS_PATH)

gs://kds-40f526733933c01f1885087fe24fcb06ce3f5c601cd74b3d084cfe68/tfhub_models/efficientnetv2-l-21k-ft1k/feature_vector


In [24]:
if v2:
    trained_model_path = '/kaggle/working/effnetV2_model'
    if not os.path.isdir(trained_model_path):
        os.makedirs(trained_model_path)
    else:
        shutil.rmtree(trained_model_path)
    print('model path')

model path


In [25]:
ONE_FOLD = False
if v2:
    print(f"Dataset Name : {COMPETITION_NAME}")
    strategy = auto_select_accelerator() #Define TPU strategy and clear TPU - try to select TPU else GPU or CPU
    GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)
    BATCH_SIZE = strategy.num_replicas_in_sync * 8
    for i in range(5):
        print(f"Fold {i} start")        
        # Converting global config class object to a dictionary to log using Wandb
        # --- remove ---
        detail_path = '/new_resized_data/image_512/'
        valid_paths = GCS_DS_PATH + detail_path + df[df['fold'] == i]['id'] + '.png'
        train_paths = GCS_DS_PATH + detail_path + df[df['fold'] != i]['id'] + '.png'
        valid_labels = df[df['fold'] == i][label_cols].values
        train_labels = df[df['fold'] != i][label_cols].values
        
        IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
        IMS = 8 
        # image_level file 을 사용하므로 img_size = 512
        
        # train image decoder
        decoder = build_decoder(with_labels=True,
                                target_size=(IMSIZE[IMS], IMSIZE[IMS]),
                                ext='png')
        # valid build decoder
        test_decoder = build_decoder(with_labels=False,
                                     target_size=(IMSIZE[IMS], IMSIZE[IMS]),
                                     ext='png')
           
        train_dataset = build_dataset(train_paths,
                                      train_labels,
                                      bsize=BATCH_SIZE,
                                      decode_fn=decoder)
        
        valid_dataset = build_dataset(valid_paths,
                                      valid_labels,
                                      bsize=BATCH_SIZE,
                                      decode_fn=decoder,
                                      repeat=False,
                                      shuffle=False,
                                      augment=False,
                                      do_mix=False)
        
        try:
            n_labels = train_labels.shape[1]
        except:
            n_labels = 1
        print(f"\n n_labels : {n_labels} \n")
        
        # n_labels 부분은 num_classes -> for Dense output size
        
        with strategy.scope():
            model = tf.keras.Sequential([
                # Explicitly define the input shape so the model can be properly
                # loaded by the TFLite Converter (input layer 를 명시적으로 설정)
                tf.keras.layers.InputLayer(input_shape = [IMSIZE[IMS],IMSIZE[IMS],3]),
                tfhub.KerasLayer(MODEL_GCS_PATH, trainable=True),
                tf.keras.layers.Dropout(rate=0.1),
                tf.keras.layers.Dense(n_labels, activation='sigmoid')
            ])
            model.compile(optimizer = tf.keras.optimizers.Adam(),
                          loss='binary_crossentropy',
                          metrics = [tf.keras.metrics.AUC(multi_label=False)])
            model.summary()
            
        steps_per_epoch = train_paths.shape[0] // BATCH_SIZE 
        checkpoint = tf.keras.callbacks.ModelCheckpoint(f'{trained_model_path}/model{i}_2class.h5', 
                                                        save_best_only=True, 
                                                        monitor='val_loss',
                                                        mode='min')

        lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          patience=3,
                                                          min_lr=1e-6,
                                                          mode='min')
        annealing = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.0001,
                                                              first_decay_steps=10,
                                                              t_mul=1.0,
                                                              m_mul=1.0,
                                                              alpha=0.0)
        lr_callback = tf.keras.callbacks.LearningRateScheduler(annealing, verbose = True)
        
        history = model.fit(train_dataset,
                            epochs=20,
                            verbose=1,
                            callbacks=[checkpoint, lr_reducer],
                            steps_per_epoch=steps_per_epoch,
                            validation_data=valid_dataset)

        hist_df = pd.DataFrame(history.history)
        hist_df.to_csv(f"{trained_model_path}/history{i}_2class.csv")
        if ONE_FOLD:
            break
        

Dataset Name : siim-image
Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas
Fold 0 start

 n_labels : 2 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              117746848 
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 117,749,410
Trainable params: 117,236,834
Non-trainable params: 512,576
_________________________________________________________________
Epoch 1/20
76/76 [==============================] - 351s 2s/step - loss: 0.6154 - auc: 0.7431 - val_loss: 0.7437 - val_auc: 0.8416
Epoch 2/20
76/76 [==============================] - 34s 444ms/step - loss: 0.5400 - auc: 0.8125 - val_loss: 0.4348 - val

In [31]:
!zip -r new_efnV2_cutmix.zip /kaggle/working/effnetV2_model

  adding: kaggle/working/effnetV2_model/ (stored 0%)
  adding: kaggle/working/effnetV2_model/history1_2class.csv (deflated 58%)
  adding: kaggle/working/effnetV2_model/history2_2class.csv (deflated 58%)
  adding: kaggle/working/effnetV2_model/model2_2class.h5 (deflated 7%)
  adding: kaggle/working/effnetV2_model/model3_2class.h5 (deflated 7%)
  adding: kaggle/working/effnetV2_model/model0_2class.h5 (deflated 7%)
  adding: kaggle/working/effnetV2_model/history0_2class.csv (deflated 57%)
  adding: kaggle/working/effnetV2_model/history3_2class.csv (deflated 58%)
  adding: kaggle/working/effnetV2_model/history4_2class.csv (deflated 58%)
  adding: kaggle/working/effnetV2_model/model1_2class.h5 (deflated 7%)
  adding: kaggle/working/effnetV2_model/model4_2class.h5 (deflated 7%)


In [ ]:
# dhgffddfgffkfdfgf

In [ ]:
# New Data efnb7 score
# vaAUC : 0.913, 0.871, 0.876, 0.900, 0.903
# loss : 0.373, 0.414, 0.421, 0.393, 0.368

# 왜 두번째 실험이 더 나을까? seed 에 의한 우연? or augmentation (이것 말고는 차이가 딱히 없다.)
# -> 데이터 셋이 잘못되었기 때문에 검증 불가
# 왜 efnV2 는 성능이 별로인가? -> 잘 모르곘다. 이미지 사이즈가 잘 안맞아서 그런가? 768 로 맞춰줘야 하나?

In [ ]:
if v2:
    # train, valid paths for TPU form (just like this)
    print(GCS_DS_PATH)
    print('GCS_DS_PATH == load_dir\n')
    print((GCS_DS_PATH + '/image/' + df[df['fold'] == 0]['id'] + '.png').head())

# zip

In [ ]:
# zip
#!zip -r image_2class_model.zip /kaggle/working/
#!zip -r efnV2_2class_model2.zip /kaggle/working/effnetV2_model
#!zip -r new_efnb7_2class.zip /kaggle/working/effnetB7_model
!zip -r new_efnv2_xl.zip /kaggle/working/effnetV2_model

In [ ]:
from glob import glob
glob('/kaggle/working/*')
!wget -r /kaggle/working/efnV2_2class_model2.zip